In [1]:
# Author: Tiago Tamagusko (tamagusko@gmail.com)
# Version: 3.0 (2023-09-16)

In [2]:
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

In [3]:
# Data

# Structure
AGE = pd.read_csv('raw/age.csv')
THICKNESS = pd.read_csv('raw/repr_thickness.csv')
SN = pd.read_csv('raw/sn.csv') # structure number
# Traffic
AADTT = pd.read_csv('raw/aadtt.csv')
# Climate
PRECIPITATION = pd.read_csv('raw/precipitation.csv')
TEMPERATURE = pd.read_csv('raw/temperature.csv')
# Performance
IRI = pd.read_csv('raw/iri.csv')

In [4]:
# Fixing column ' ' in PRECIPITATION dataset
PRECIPITATION = PRECIPITATION.copy()
PRECIPITATION.rename(columns={' ': 'STATE_CODE'}, inplace=True)

In [5]:
# Cleaning data
AGE = AGE[['STATE_CODE', 'SHRP_ID', 'TRAFFIC_OPEN_DATE']]
AGE['TRAFFIC_OPEN_DATE'] = pd.DatetimeIndex(AGE['TRAFFIC_OPEN_DATE']).year
AGE['TRAFFIC_OPEN_DATE'] = AGE['TRAFFIC_OPEN_DATE'].astype("Int64")
THICKNESS = THICKNESS[['STATE_CODE', 'SHRP_ID', 'REPR_THICKNESS']]
SN = SN[['STATE_CODE', 'SHRP_ID', 'SN_VALUE']]
AADTT = AADTT[['STATE_CODE', 'SHRP_ID', 'YEAR', 'AADTT_ALL_TRUCKS_TREND']]
PRECIPITATION = PRECIPITATION[['STATE_CODE', 'SHRP_ID', 'YEAR', 'TOTAL_ANN_PRECIP']]
TEMPERATURE = TEMPERATURE[['STATE_CODE', 'SHRP_ID', 'YEAR', 'MEAN_ANN_TEMP_AVG']]
IRI = IRI[['STATE_CODE', 'SHRP_ID', 'VISIT_DATE', 'MRI']]
IRI['VISIT_DATE'] = pd.DatetimeIndex(IRI['VISIT_DATE']).year
IRI.rename(columns={'VISIT_DATE': 'YEAR'}, inplace=True)

In [6]:
average_IRI = IRI.groupby(['STATE_CODE', 'SHRP_ID', 'YEAR'])['MRI'].mean().reset_index()
average_THICKNESS = THICKNESS.groupby(['STATE_CODE', 'SHRP_ID'])['REPR_THICKNESS'].mean().reset_index()

In [7]:
# Creating a unified dataset
DATA = AADTT.copy()
DATA = pd.merge(DATA, AGE, on=['SHRP_ID', 'STATE_CODE'], how='right')
DATA = pd.merge(DATA, average_THICKNESS, on=['SHRP_ID', 'STATE_CODE'], how='right')
DATA = pd.merge(DATA, SN, on=['SHRP_ID', 'STATE_CODE'], how='right')
DATA = pd.merge(DATA, PRECIPITATION, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='right')
DATA = pd.merge(DATA, TEMPERATURE, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='right')
DATA = pd.merge(DATA, average_IRI, on=['SHRP_ID', 'STATE_CODE', 'YEAR'], how='right')
DATA.shape

(395, 10)

In [8]:
# STATION_ID = STATE_CODE _SHRP_ID
DATA['STATION_ID'] = DATA['STATE_CODE'].astype(str) + "_" + DATA['SHRP_ID'].astype(str)
DATA = DATA.drop(['STATE_CODE', 'SHRP_ID'], axis=1)

In [9]:
# Sort dataset
DATA = DATA.sort_values(['YEAR'], ascending=[True])

In [10]:
DATA.columns

Index(['YEAR', 'AADTT_ALL_TRUCKS_TREND', 'TRAFFIC_OPEN_DATE', 'REPR_THICKNESS',
       'SN_VALUE', 'TOTAL_ANN_PRECIP', 'MEAN_ANN_TEMP_AVG', 'MRI',
       'STATION_ID'],
      dtype='object')

In [11]:
# Rename columns
DATA = DATA.rename({'AADTT_ALL_TRUCKS_TREND': 'AADTT',
                    'REPR_THICKNESS': 'THICKNESS',
                    'SN_VALUE': 'SN',
                    'MRI': 'IRI',
                    'TOTAL_ANN_PRECIP': 'PRECIPITATION',
                    'MEAN_ANN_TEMP_AVG': 'TEMPERATURE',
                    }, axis=1)

In [12]:
DATA.head()

,YEAR,AADTT,TRAFFIC_OPEN_DATE,THICKNESS,SN,PRECIPITATION,TEMPERATURE,IRI,STATION_ID
244,1989,1190.0,1985,140.566667,6.1,1372.800049,12.000000,0.8984,34_1638
136,1989,490.0,1985,261.600000,3.3,260.799988,6.100000,1.2738,16_1021
233,1989,151.0,1982,164.475000,4.3,634.200012,6.000000,1.0314,30_1001
277,1989,195.0,1984,110.500000,3.2,1546.099976,16.200001,1.0448,37_1030
166,1989,1690.0,1985,268.600000,8.3,1125.199951,7.200000,0.7238,23_1012


In [13]:
DATA.describe().T

,count,mean,std,min,25%,50%,75%,max
YEAR,395.0,1995.612658,4.974998,1989.0,1992.0,1994.0,1999.0,2016.0
AADTT,395.0,471.632911,539.745625,11.0,156.0,278.0,536.0,3126.0
TRAFFIC_OPEN_DATE,395.0,1980.321519,5.449856,1970.0,1975.0,1983.0,1984.0,1989.0
THICKNESS,395.0,230.405278,212.553605,64.45,109.225,163.825,261.6,1177.06
SN,395.0,4.627089,1.554672,2.0,3.6,4.2,5.7,8.3
PRECIPITATION,395.0,1006.025572,471.496033,92.199997,631.899994,1059.800049,1363.850036,2091.0
TEMPERATURE,395.0,14.613418,6.046667,2.7,8.5,15.8,18.299999,25.9
IRI,395.0,1.370352,0.598253,0.6208,0.9203,1.176,1.6814,4.0048


In [14]:
# verify data
DATA.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 395 entries, 244 to 220
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   YEAR               395 non-null    int64  
 1   AADTT              395 non-null    float64
 2   TRAFFIC_OPEN_DATE  395 non-null    Int64  
 3   THICKNESS          395 non-null    float64
 4   SN                 395 non-null    float64
 5   PRECIPITATION      395 non-null    float64
 6   TEMPERATURE        395 non-null    float64
 7   IRI                395 non-null    float64
 8   STATION_ID         395 non-null    object 
dtypes: Int64(1), float64(6), int64(1), object(1)
memory usage: 31.2+ KB


In [15]:
# Save data
DATA.to_csv('processed/ltpp_data.csv', index=None, header=True)